In [1]:
import glob
import pandas as pd
from math import isnan
from copy import deepcopy as copy
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline  

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
import os
path = os.getcwd() + '/data' # use your path
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
frame = pd.concat(list_)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (10,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,15,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [32]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords 
import string
lemma = WordNetLemmatizer()
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 

def clean(doc):
    # input: string
    # output: string
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude or ch == "!")
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split()).split()
    link_free = [word for word in normalized if 'http' not in word]
    cleaned = " ".join(word for word in link_free)
    
    return cleaned

In [33]:
import nltk
import string as st
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
# getting sentiment of sentence
def sentiment(tweets):
    # input: list of strings
    # output: list of ints
    sentiments = []
    for tweet in tweets:
        tweet = clean(tweet)
        score = sid.polarity_scores(tweet)
        sentiments.append(score['compound'])
    return sentiments

In [53]:
df = frame[frame['language'] == 'English']
from tqdm import tqdm
tweets = {}
sentiments = {}
for category in df['account_category'].unique():
    print(category)
    dfc = df[df['account_category'] == category]
    g = dfc.groupby(by = dfc['external_author_id']).groups
    tweets[category] = {}
    sentiments[category] = {}
    for user in tqdm(g):
        tweets[category][user] = [tweet for tweet in df.iloc[g[user]]['content'].values if '\t' not in tweet]
        sentiments[category][user] = sentiment(tweets[category][user])

RightTroll


100%|████████████████████████████████████████████████████████████████████████████████| 449/449 [04:49<00:00,  1.60it/s]


Fearmonger


100%|████████████████████████████████████████████████████████████████████████████████| 125/125 [00:05<00:00, 22.90it/s]


NonEnglish


...

100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [01:23<00:00,  1.36it/s]


NewsFeed


100%|██████████████████████████████████████████████████████████████████████████████████| 53/53 [03:23<00:00,  3.83s/it]


Commercial


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:40<00:00,  4.64s/it]


In [62]:
import csv
with open('full_troll_tweet_sentiments.tsv', 'w', encoding='utf-8') as f:
    f.write('category\ttweets\tsentiments\n')
    for category in tweets:
        print(category)
        limiter = len(tweets[category]) // 4
        for user,_ in tqdm(list(tweets[category].items())[:limiter]):
            f.write(category+'\t[')
            line = ''
            for tweet in tweets[category][user]:
                line += '\"'+tweet.replace('"','')+'\"'+','
            if len(line) > 0 and line[-1] == ',':
                line = line[:-1]
            f.write(line)
            f.write(']\t'+str(sentiments[category][user]) + '\n')

RightTroll


100%|████████████████████████████████████████████████████████████████████████████████| 112/112 [00:03<00:00, 34.04it/s]


Fearmonger


100%|████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 1506.24it/s]


NonEnglish


...

100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [00:00<00:00, 30.56it/s]


NewsFeed


100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:59<00:00,  3.67s/it]


Commercial


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:10<00:00,  4.51s/it]
